In [2]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [3]:
path = DATA_DIR/'cairns_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
#path = DATA_DIR/'other_feeds'/'seq_20170310_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


date 20140526


In [30]:
feed.validate()

,type,message,table,rows
0,warning,Feed expired,calendar/calendar_dates,[]
1,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[17710, 17722, 17730, 17732, 17736, 17739, 177..."


In [37]:
import zipfile

    
def summarize(feed, table=None):
    """
    """
    if table is not None:
        tables = [table]
    else:
        tables = gt.GTFS_REF.table.unique()
    
    frames = []
    for table in tables:
        f = getattr(feed, table)
        if f is None:
            continue
        
        def my_agg(col):
            d = {}
            d['column'] = col.name
            d['#values'] = col.size
            d['#nonnull_values'] = col.count()
            d['#unique_values'] = col.nunique()
            d['min_value'] = col.dropna().min()
            d['max_value'] = col.dropna().max()
            return pd.Series(d)
        
        g = f.apply(my_agg).T.reset_index(drop=True)
        g['table'] = table
        frames.append(g)
        
    cols = ['table', 'column', '#values', '#nonnull_values', '#unique_values', 
      'min_value', 'max_value']
    
    if not frames:
        f = pd.DataFrame(columns=cols)
    else:
        f = pd.concat(frames)
        # Rearrange columns
        f = f[cols].copy()

    return f

# Test some
list_gtfs(path)
summarize(feed)

,table,column,#values,#nonnull_values,#unique_values,min_value,max_value
0,agency,agency_name,1,1,1,Department of Transport and Main Roads - Trans...,Department of Transport and Main Roads - Trans...
1,agency,agency_url,1,1,1,http://www.sunbus.com.au,http://www.sunbus.com.au
2,agency,agency_timezone,1,1,1,Australia/Brisbane,Australia/Brisbane
3,agency,agency_lang,1,1,1,en,en
4,agency,agency_phone,1,1,1,(07)40576411,(07)40576411
0,calendar,service_id,4,4,4,CNS2014-CNS_MUL-Saturday-00,CNS2014-CNS_MUL-Weekday-00-0000100
1,calendar,monday,4,4,2,0,1
2,calendar,tuesday,4,4,2,0,1
3,calendar,wednesday,4,4,2,0,1
4,calendar,thursday,4,4,2,0,1


In [15]:
s = feed.routes.route_type
s.count()

22

In [60]:
feed.stops

,stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,0005AMB,NaN,Ambassadeur,-1.285963,36.826048,NaN,NaN,1,
1,0006BSS,NaN,Bus Station,-1.287191,36.828881,NaN,NaN,1,
2,0007COM,NaN,Commercial,-1.284282,36.826188,NaN,NaN,1,
3,0512BST,NaN,Bus Station,-1.287281,36.828863,NaN,NaN,,0006BSS
4,0512BSN,NaN,Bus Station,-1.287635,36.829215,NaN,NaN,,0006BSS
5,0512BSL,NaN,Bus Station,-1.287229,36.829477,NaN,NaN,,0006BSS
6,0512CML,NaN,Commercial Imara,-1.284306,36.826257,NaN,NaN,,0007COM
7,0512CMC,NaN,Commercial,-1.283947,36.826437,NaN,NaN,,0007COM
8,0512AMU,NaN,Ambassadeur,-1.285315,36.825615,NaN,NaN,,
9,0512ESC,NaN,Easy Coach/Uchumi,-1.289218,36.828251,NaN,NaN,,


In [10]:
feed.stop_times


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,8669735,7:09:00,7:09:00,55,1,NaN,0,0,NaN
1,8669735,7:10:50,7:10:50,56,2,NaN,0,0,0.3092
2,8669735,7:12:00,7:12:00,57,3,NaN,0,0,0.5052
3,8669735,7:13:14,7:13:14,58,4,NaN,0,0,0.7143
4,8669735,7:14:20,7:14:20,59,5,NaN,0,0,0.9004
5,8669735,7:15:00,7:15:00,60,6,NaN,0,0,1.0071
6,8669735,7:15:39,7:15:39,61,7,NaN,0,0,1.1881
7,8669735,7:16:20,7:16:20,62,8,NaN,0,0,1.3772
8,8669735,7:16:41,7:16:41,63,9,NaN,0,0,1.4773
9,8669735,7:17:21,7:17:21,64,10,NaN,0,0,1.6603


In [ ]:
g.groupby(['route_id', 'orientation']).count()